In [ ]:
from shapely import geometry
from pathlib import Path
from importlib import reload
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed

import geopandas as gpd
import numpy as np
import os
import warnings
import pandas as pd


import roads as rd
rd = reload(rd)

# Ideas

* Use some sort of structuring element - circle, square etc - and move across road network and count how many times it does not intersect any road. Use a pretty small one, like 1 km^2

1. Generate a bunch of small structuring elements
2. count how many intersect the full-state roads

In [ ]:
def run_analysis_for_state(
    statename, 
    blocksize,
    chunksize=400,
    max_workers=6
):
    state_data = rd.StateData(statename)
    
    print(f'Opening shapefiles for {statename}')
    state_roads_file = state_data.statewide_file_path('prisecroads', '.shp')
    state_edge_file = state_data.statewide_file_path('cd118', '.shp')
    
    state_map = rd.StateMaps(state_roads_file, state_edge_file)
    
    time_blocks_st = datetime.now()
    print(f'Making {blocksize}m x {blocksize}m blocks')
    state_map.make_blocks(blocksize)
    time_blocks_en = datetime.now()
    print(f'making blocks took {time_blocks_en-time_blocks_st}')
    
    time_inter_st = datetime.now()
    print(f'Running intersection analysis with chunksize {chunksize} ({state_map.n_blocks // chunksize} chunks) and {max_workers} workers')
    n_ints, a_ints = state_map.get_intersections(chunksize=chunksize, max_workers=max_workers)
    time_inter_en = datetime.now()
    print(f'running intersection analysis took {time_inter_en-time_inter_st}')
    
    print(f'Number intersections: {n_ints}')
    print(f'Percent blocks with intersections: {n_ints / state_map.n_blocks}')
    print(f'Area of blocks with intersections: {a_ints}')
    print(f'Percent of area with intersections: {a_ints / state_map.area}')
    
    return state_map

## Results
* New York: 14 minutes - 27%
* Nevada: 5 minutes? - 5.5%
* Massachusetts: 2 minutes - 37%

In [ ]:
run_analysis_for_state('new york', 2000)

In [ ]:
run_analysis_for_state('massachusetts', 2000)